In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
ENTREPOT_PATH = '~/Bureau/utils/data/'
METEO_PATH = '~/Bureau/utils/data/meteo/'
SPATIAL_PATH = './data/spatial/'

In [49]:
df = {}

def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
    'noeuds_synthetise', 'connection_synthetise', 'synthetise', 'culture', 'noeuds_synthetise_restructure'
]

tables_performance = [
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',',index_col='id', verbose=False)

# import des données de performance
#import_dfs(['intervention_realise_agrege', 'combinaison_outil_materiel'], ENTREPOT_PATH, sep = ',', verbose=False)


100%|██████████| 5/5 [00:01<00:00,  4.44it/s]


In [67]:
studied_synthetise_id = [
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_000f911f-5e67-4280-ae77-35098c17aa5d', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_00b5a0b4-39a6-4822-802b-e81fd44386a2', 
    #'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_05e5d9d6-ad2b-43b2-b37a-1f32850e37a1'
]

In [68]:
df['synthetise_test'] = df['synthetise'].loc[
    df['synthetise'].index.isin(studied_synthetise_id)
]

In [69]:
df['noeuds_synthetise_test'] = df['noeuds_synthetise'].loc[
    df['noeuds_synthetise']['synthetise_id'].isin(df['synthetise_test'].index)
]

In [70]:
df['connection_synthetise_test'] = df['connection_synthetise'].loc[
    (df['connection_synthetise']['source_noeuds_synthetise_id'].isin(df['noeuds_synthetise_test'].index)) |
    (df['connection_synthetise']['cible_noeuds_synthetise_id'].isin(df['noeuds_synthetise_test'].index))
]

In [71]:
left = df['noeuds_synthetise']
right = df['noeuds_synthetise_restructure']
df['noeuds_synthetise_extanded'] = pd.merge(left, right, left_index=True, right_index=True, how='left')

In [72]:
left = df['noeuds_synthetise_extanded']
right = df['culture']
df['noeuds_synthetise_extanded'] = pd.merge(left, right, left_on='culture_id', right_index=True, how='left')


In [73]:
left = df['connection_synthetise_test']
right = df['noeuds_synthetise_extanded'].rename(columns={'nom' : 'nom_culture_cible'})
merge = pd.merge(left, right, left_on='cible_noeuds_synthetise_id', right_index=True, how='left')

left = merge
right = df['noeuds_synthetise_extanded'][['nom']].rename(columns={'nom' : 'nom_culture_source'})
merge = pd.merge(left, right, left_on='source_noeuds_synthetise_id', right_index=True, how='left')

In [74]:
merge[['rang', 'nom_culture_source', 'nom_culture_cible', 'synthetise_id']]

,rang,nom_culture_source,nom_culture_cible,synthetise_id
id,,,,
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_c64570ec-7563-47b6-afaf-13c4a71727fe,1,prairie temporaire,Maïs,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_439cd55e-fa78-455a-9504-234279059142,2,Maïs,Maïs,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_65c9c876-9634-4542-aa00-0ed70d770650,3,Maïs,Orge d'hiver,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_fc6dff67-16ea-4cdf-ab0f-4b5ade4d49e4,3,Maïs,Triticale,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_9e756084-e6e0-42f1-a007-9b2e7c625555,0,Triticale,prairie temporaire,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_4fc23bd1-5986-4abb-8968-0f485b8175e0,0,Orge d'hiver,prairie temporaire,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_cffd990a-b401-46d8-b338-28ad52c749ea,1,permanent grassland,permanent grassland,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_aaef3477-1e6b-4226-ba44-56006336166a,0,permanent grassland,permanent grassland,fr.inra.agrosyst.api.entities.practiced.Practi...


In [ ]:
path=''
df['connection_synthetise_test'].to_csv(path+'connection_synthetise'+'.csv')
df['noeuds_synthetise_test'].to_csv(path+'noeuds_synthetise'+'.csv')
